### 从ModelScope下载模型到本地

这里推荐创建一个conda环境，方便使用pycharm进行解释器的配置。

In [ ]:
!conda create -n qwen python=3.11 # 推荐使用3.11版本的python，3.12版本的尝试过不兼容，transformers库版本不兼容等问题
!conda activate qwen # 激活环境

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh # 安装rust环境
!source "$HOME/.cargo/env.fish"
!rustc --version
!reboot

In [ ]:
!pip install modelscope transformers==4.32.0 accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed

In [2]:
from modelscope import snapshot_download
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_dir = snapshot_download('qwen/Qwen-1_8B-Chat')

tokenizer = AutoTokenizer.from_pretrained(
    model_dir, 
    trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-05-13 15:09:10,989 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Downloading: 100%|██████████████████████████████████████████████████████████████████| 54.3k/54.3k [00:00<00:00, 821kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 15.0k/15.0k [00:00<00:00, 430kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 237k/237k [00:00<00:00, 1.62MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████| 116k/116k [00:00<00:00, 962kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████| 2.44M/2.44M [00:00<00:00, 6.64MB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 473k/473k [00:00<00:00, 2.45MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 14.3k/14.3k [00:00<00:00, 361kB/s]
Downloading: 100%|███████████████████████████████████

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-05-13 15:15:33,897 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


In [11]:
print(model_dir) # 模型目录

/root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat


In [ ]:
!mv /root/.cache/modelscope/hub/qwen /root/QwenLLM/qwen # 移动模型到指定目录

### 模型运行测试

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
from modelscope import GenerationConfig

model_dir = '/root/QwenLLM/qwen/Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, fp16=True).eval()
model.generation_config = GenerationConfig.from_pretrained(model_dir, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参

In [ ]:
response, history = model.chat(tokenizer, "你好", system='使用秦始皇的语气和我说话，要显示出始皇帝的威严', history=None)
print(response)

In [ ]:
response, history = model.chat(tokenizer, "陛下为什么能够统一六国？", system='使用秦始皇的语气和我说话，要显示出始皇帝的威严', history=history)
print(response)